TODO - DESCRIBE THIS FILE 

In [2]:
# Install Pillow if not already installed
# %pip install pillow  # This line is likely for a Jupyter Notebook environment

# Standard Library
import json
import math
import os
import shutil
from typing import Any, NewType
import csv 

# Third-Party Libraries
from PIL import Image, ImageDraw
from PIL.TiffTags import TAGS

# Type Alias for Bounding Boxes
Box = NewType('Box', tuple[int, int, int, int])

# Dataset Paths (Remember to replace placeholders with your actual paths)
DATASET_SOURCE = "/mnt/f/dataUtils/raw_data"  # FIXME: Change accordingly
OUTPUT_DATASET = "/mnt/e/dataset"  # FIXME: Change accordingly

Math utils

In [3]:
#utils
def get_image_metadata(img: Image) -> dict[str, float]:
    """Extracts resolution metadata from a TIFF image.
    Args:
        img: A PIL Image object.
    Returns:
        A dictionary containing resolution information (x, y, and unit).
    """
    tiff_tags = {TAGS.get(tag, tag): value for tag, value in img.tag.items()}
        
    res = {
        "x_resolution": tiff_tags['XResolution'][0][0]/tiff_tags['XResolution'][0][1],
        "y_resolution": tiff_tags['YResolution'][0][0]/tiff_tags['YResolution'][0][1],
        "resolution_unit": tiff_tags['ResolutionUnit'] 
    }
    return res

class ImageUtils:
    """Provides utility functions for working with image resolutions and areas."""

    def __init__(self, img):
        self.img = img
        self.metadata = get_image_metadata(img)
        self.x_resolution = float(self.metadata["x_resolution"])
        self.y_resolution = float(self.metadata["y_resolution"])

    def area_units_to_pixels(self, area):
        """Converts area from resolution units to pixels."""
        return float(area) * (max(self.x_resolution, self.y_resolution) ** 2)

    def area_pixels_to_units(self, area):
        """Converts area from pixels to resolution units."""
        return float(area) / (max(self.x_resolution, self.y_resolution) ** 2)


class ImageUtilOpener:
    """Context manager for opening and working with an image."""

    def __init__(self, file_name):
        self.file_name = file_name

    def __enter__(self):
        self.img = Image.open(self.file_name)
        return ImageUtils(self.img)

    def __exit__(self, *args):
        self.img.close()


Read csv+tif ImageJ-created dataset, and convert to dict
of images_path as keys and corresponding bounding boxes

In [ ]:
def convert_cxcywh_to_xyxy(box: Box) -> Box:
    """Converts a bounding box from center-x, center-y, width, height format (CXCYWH) 
    to top-left-x, top-left-y, bottom-right-x, bottom-right-y format (XYXY).
    Args:
        box: A tuple representing the bounding box in CXCYWH format.
    Returns:
        A tuple representing the bounding box in XYXY format.
    """
    cx, cy, w, h = box
    xmin = cx-w//2
    ymin = cy-h//2
    xmax = cx+(w+1)//2  # ceil
    ymax = cy+(h+1)//2  # ceil
    return xmin, ymin, xmax, ymax


def sanitize_annotations(name:str, annotation: dict[str, Any]) -> None:
    """Sanitizes annotations by removing duplicates and invalid bounding boxes.

    Args:
        name: The name of the image associated with the annotations.
        annotations: A dictionary containing "category_id" and "boxes" lists.

    Returns:
        A dictionary containing sanitized "category_id" and "boxes" lists.
    """
    sanitized_annotations = {
        "category_id": [],
        "boxes": []
    }
    
    bbox_dict = dict()
    for label, bbox in zip(annotation["category_id"], annotation["boxes"]):
        if bbox in bbox_dict:
            if bbox_dict[bbox] == label:
                print(f"WARNING: Duplicate bbox found in {name}: {bbox}")
            else:
                print(f"ERROR: Same bbox with different label found in {name}: {bbox}")
            continue
        if any(coord < 0 for coord in bbox):
            print(f"Corrupted box found in {name}: {bbox}")
            continue

        bbox_dict[bbox] = label

    for key in bbox_dict:
        sanitized_annotations["category_id"].append(bbox_dict[key])
        sanitized_annotations["boxes"].append(key)

    return sanitized_annotations
        
def convert_annotations(csv_path: str, img_path: str) -> dict[str, Any]:
    """Converts a CSV file and an image into annotations.
    Args:
        csv_path: The path to the CSV file containing annotations.
        img_path: The path to the image file.
    Returns:
        A dictionary containing "category_id" and "boxes" lists representing the annotations.
    """
    annotations = {}
    name = os.path.basename(img_path).removesuffix(".tif")
    annotations = {
        "category_id": [],
        "boxes": []
    }
    
    with ImageUtilOpener(img_path) as util:
        if os.path.exists(csv_path): 
            with open(csv_path) as data_file:
                data = csv.reader(data_file)
                next(data) # Skip header row
                for row in data:
                    try:
                        _, label, area, category_id, _ = row  # Unpack row, ignore filename and category_name
                        _, (y, x) = label.split(':')  # Extract coordinates from label
                        y, x = int(y), int(x)

                        area_in_pixels = util.area_units_to_pixels(float(area))
                        bbox_side = int(math.sqrt(area_in_pixels / math.pi) * 2)  # Calculate square side length

                        bbox = convert_cxcywh_to_xyxy((x, y, bbox_side, bbox_side))
                        annotations["category_id"].append(int(category_id))
                        annotations["boxes"].append(bbox)
                    except Exception as e:
                        annotations["boxes"] = []
                        annotations["category_id"] = []
                        print(f"Error processing row in {csv_path}: {e}")
                        print(f"{csv_path} Failed")
                        break  
                     
    return annotations
    
    
def convert_dataset(data_path: str, imgs_path: str, res_path: str, replace_imgs=True):
    """Converts a dataset from the source format to the desired output format.
    Args:
        data_path: The path to the directory containing CSV annotation files
        imgs_path: The path to the directory containing image files
        res_path: The path to the output directory where the converted dataset will be saved
        replace_imgs: Whether to replace existing images in the output directory
    """
    img_list = [
        os.path.splitext(f)[0] 
        for f in os.listdir(imgs_path) 
        if os.path.isfile(os.path.join(imgs_path, f)) and f.endswith(".tif")
    ]
    
    if replace_imgs:
        try:
            shutil.rmtree(res_path)
        except FileNotFoundError:
            pass

    os.makedirs(os.path.join(res_path, "imgs"), exist_ok=True)
       
    dataset = {
        "imgs": [],
        "annotations": []
    }
    img_id = 1  # Start image IDs from 1
    
    for name in img_list:
        csv_path = f'{data_path}/{name}.tif.csv'
        img_path = f'{imgs_path}/{name}.tif'
        annotations = convert_annotations(csv_path, img_path)
        if not annotations["boxes"]:
            continue # Skip images without annotations

        if replace_imgs:
            img = Image.open(img_path)
            img.save(os.path.join(res_path, "imgs", f"{img_id}.jpeg"), quality=100)
        
        annotations = sanitize_annotations(name, annotations)
        dataset["imgs"].append(f"{img_id}")
        dataset["annotations"].append(annotations)
        img_id += 1

    with open(os.path.join(res_path, "dataset.json"), "w") as outfile:
        json.dump(dataset, outfile, indent=4)
    print("Finished converting")
    
convert_dataset(f"{DATASET_SOURCE}/csv", f"{DATASET_SOURCE}/imgs", OUTPUT_DATASET, replace_imgs=False)

Patch dataset into images of desired size

In [5]:
def calculate_cropped_coefficient(src_box: Box, cover_box: Box) -> float:
    """Calculates the cover coefficient between two bounding boxes.

    The cover coefficient measures how much of the `src_box` is covered by the `cover_box`.
    It is calculated as the intersection area between the two boxes divided by the area of the `src_box`.

    Args:
        src_box: The source bounding box in XYXY format.
        cover_box: The covering bounding box in XYXY format.

    Returns:
        The cover coefficient between the two boxes (a value between 0 and 1).
    """
    xmin1, ymin1, xmax1, ymax1 = src_box
    xmin2, ymin2, xmax2, ymax2 = cover_box
    inter_xmin = max(xmin1, xmin2)
    inter_ymin = max(ymin1, ymin2)
    inter_xmax = min(xmax1, xmax2)
    inter_ymax = min(ymax1, ymax2)
    
    intersection_area = max(0, inter_xmax - inter_xmin) * max(0, inter_ymax - inter_ymin)
    src_box_area = max(0, xmax1 - xmin1) * max(0, ymax1 - ymin1)
    if src_box_area == 0:
        return -1
    
    return 1 - intersection_area / src_box_area
    

def crop(img: Image.Image,
         crop_box: Box,
         annots: tuple[list[int], list[Box]],
         crop_tolerance: float,
         crop_not_tolerated: bool = True) -> tuple[Image.Image, tuple[list[int], list[Box]]]:
    """Crops an image and adjusts annotations based on the crop region.

    Args:
        img: The PIL Image object to crop
        crop_box: The bounding box defining the crop region in XYXY format
        annots: A dictionary containing "category_id" and "boxes" lists representing annotations
        crop_tolerance: The tolerance for considering an annotation as fully within the crop
        crop_not_tolerated: Whether to draw over partially cropped annotations on the image

    Returns:
        A tuple containing the cropped image and the adjusted annotations
    """
    cropped_img = img.crop(crop_box)
    cropped_annots = {"category_id": [], "boxes": []}
    labels, bboxes = annots["category_id"], annots["boxes"]
    
    draw_context = ImageDraw.Draw(cropped_img)
    for label, bbox in zip(labels, bboxes):
        relative_bbox = (max(crop_box[0], bbox[0]) - crop_box[0],
                         max(crop_box[1], bbox[1]) - crop_box[1],
                         min(crop_box[2], bbox[2]) - crop_box[0],
                         min(crop_box[3], bbox[3]) - crop_box[1])
        cropped = calculate_cropped_coefficient(crop_box, bbox)
        if cropped <= crop_tolerance:
            cropped_annots["category_id"].append(label)
            cropped_annots["boxes"].append(relative_bbox)
        elif cropped > 0 and crop_not_tolerated:
            draw_context.rectangle(relative_bbox, width=1, fill="purple")

    return cropped_img, cropped_annots

def patch_img(img: Image.Image,
          annots: dict[str, Any],
          desired_image_size: int,
          overlap: float,
          crop_tolerance: float) -> tuple[list[Image.Image], list[tuple[list[int], list[Box]]]]:
    """Patches a large image into smaller images with adjusted annotations.

    Args:
        img: The PIL Image object to patch
        annots: A tuple containing category_id and boxes lists representing annotations
        desired_image_size: The desired size of each patch
        overlap: The overlap between adjacent patches (as a fraction of `desired_image_size`)
        crop_tolerance: The tolerance for considering an annotation as fully within a patch
    Returns:
        A tuple containing a list of patched images and a list of corresponding adjusted annotations
    """
    
    overlap_size = int(desired_image_size * overlap)
    no_overlap_size = desired_image_size - overlap_size
    
    imgs_per_width = math.ceil(float(img.width) / no_overlap_size)
    imgs_per_height = math.ceil(float(img.height) / no_overlap_size)

    padded_height = imgs_per_width * no_overlap_size + overlap_size
    padded_width = imgs_per_width * no_overlap_size + overlap_size

    padded_img = Image.new("RGB", (padded_width, padded_height))
    padded_img.paste(img)

    patched_imgs, patched_annots = [], []

    for row in range(imgs_per_height):
        for col in range(imgs_per_width):
            xmin, ymin = col * no_overlap_size, row * no_overlap_size
            xmax, ymax = xmin + desired_image_size, ymin + desired_image_size

            crop_box = (xmin, ymin, xmax, ymax)

            cropped_img, cropped_annots = crop(padded_img, crop_box, annots, crop_tolerance)

            if cropped_annots["category_id"]:
                patched_imgs.append(cropped_img)
                patched_annots.append(cropped_annots)

    return patched_imgs, patched_annots

def patch_dataset(dataset_root: str,
                  desired_image_size: int = 1024,
                  overlap: float = 0.2,
                  crop_tolerance: float=0.3):
    """Patches images in a dataset and saves the patched images and annotations.

    Args:
        dataset_root: The root directory of the dataset
        desired_image_size: The desired size of each patch
        overlap: The overlap between adjacent patches (as a fraction of `desired_image_size`)
        crop_tolerance: The tolerance for considering an annotation as fully within a patch
    """
    dataset_root = os.path.normpath(dataset_root)
    annot_file = os.path.join(dataset_root, "dataset.json")
    imgs_dir = os.path.join(dataset_root, "imgs")
    dataset_parent = os.path.dirname(dataset_root)
    patched_root = os.path.join(dataset_parent, os.path.basename(dataset_root) + "_patched")
    os.makedirs(os.path.join(patched_root, "imgs"), exist_ok=True)
    
    patched_dataset = {
        "imgs": [],
        "annotations": []
    }
    
    with open(annot_file, 'r') as annot_file:
        dataset = json.load(annot_file)
        
    image_id = 1
    for img_name, annotations in zip(dataset["imgs"], dataset["annotations"]):
        img = Image.open(os.path.join(imgs_dir, f"{img_name}.jpeg"))
        patched_imgs, patched_annots = patch_img(img, annotations,desired_image_size, overlap, crop_tolerance)
        for img, annots in zip(patched_imgs, patched_annots):
            patched_dataset["imgs"].append(f"{image_id}")
            patched_dataset["annotations"].append(annots)
            img.save(os.path.join(patched_root, "imgs", f"{image_id}.jpeg"), quality=100)
            image_id += 1
            
    with open(os.path.join(patched_root, "dataset.json"), "w") as outfile: 
        json.dump(patched_dataset, outfile, indent=4)
        
patch_dataset(OUTPUT_DATASET, crop_tolerance=0) 